# example 3 on Iris dataset using pytorch - (hoạt động tốt)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [ ]:
# Load the Iris dataset
iris = load_iris()
data = iris.data

# Standardize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Create PyTorch dataset
dataset = TensorDataset(torch.tensor(data, dtype=torch.float32))
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, output_dim),
        )

    def forward(self, x):
        return self.model(x)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Initialize the models
input_dim = 4
latent_dim = 8

generator = Generator(latent_dim, input_dim)
discriminator = Discriminator(input_dim)

# Loss function and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

# Labels
real_label = 1.
fake_label = 0.

# Training loop
num_epochs = 5000

In [ ]:
for epoch in range(num_epochs):
    for i, (real_data,) in enumerate(dataloader):
        batch_size = real_data.size(0)

        # Train Discriminator with real data
        optimizer_D.zero_grad()
        real_output = discriminator(real_data)
        real_loss = criterion(real_output, torch.full((batch_size, 1), real_label))
        real_loss.backward()

        # Train Discriminator with fake data
        noise = torch.randn(batch_size, latent_dim)
        fake_data = generator(noise)
        fake_output = discriminator(fake_data.detach())
        fake_loss = criterion(fake_output, torch.full((batch_size, 1), fake_label))
        fake_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        fake_output = discriminator(fake_data)
        generator_loss = criterion(fake_output, torch.full((batch_size, 1), real_label))
        generator_loss.backward()
        optimizer_G.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{num_epochs}]  Loss D: {real_loss.item() + fake_loss.item()}, Loss G: {generator_loss.item()}')

In [ ]:
# Generate some fake data after training
noise = torch.randn(10, latent_dim)
fake_data = generator(noise)
fake_data = scaler.inverse_transform(fake_data.detach().numpy())
print(fake_data)